## Parameter tuning with `PredictTutor` class.

Questions for benchmark:
1. Convergence speed to optimal point. Equlibrium
2. Portfolio surrogates. Cons and pons based on convergence speed
3. Convergence and evaluations cost
4. What point select to predictions? Intersection with already evaluated points and predicted population. Maybe select point with high uncertainty

This class assumes the next point or set of points to check.
- Multi and single objective
- Models portfolio
- If there is no correct model takes a point from the sampling plan.
- Random or Sobol sequence sampling plan.

> "For 800 years have I trained Jedi." ―Yoda, to Luke Skywalker

In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')

# --- Dependencies
import pygmo as pg
import numpy as np
import pandas as pd
# import dovpanda

import plotly
from plotly import graph_objs as go
from plotly import tools
import plotly.express as px

import sklearn.gaussian_process as gp
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from src.composite import PredictTutor, ModelsUnion
from src.generator import SamplesGenerator
from src.ploting import plot_mo

from src.hypothesis.tpot_estimator import TpotWrp
from src.hypothesis.custom_gp_kernel import KERNEL_MAUNA, KERNEL_SIMPLE, KERNEL_GPML

def make_nd_pop(pro, x,y):
    nd_front = pg.fast_non_dominated_sorting(y)[0][0]
    nd_x = x[nd_front]
    nd_y = y[nd_front]
    t_pop = pg.population(pro)
    for i, p_vector in enumerate(nd_x):
        t_pop.push_back(x=p_vector, f=nd_y[i])
    return t_pop

### Problem definition

In [2]:
DIM = 2
ID = 4
OBJ = 2

# --- Problem
udp = pg.wfg(prob_id=ID, dim_dvs=DIM, dim_obj=OBJ, dim_k=OBJ-1)
# udp = pg.zdt(prob_id=ID, param=DIM)
pro = pg.problem(udp)

### Initialization a models for portfolio

In [3]:
# 1
tea_pot = TpotWrp(generations=2, population_size=10, random_state=42) 
# 2
gp_mauna = gp.GaussianProcessRegressor(kernel=KERNEL_MAUNA, alpha=0, n_restarts_optimizer=20, normalize_y=True) 
# 3
grad_uni = ModelsUnion(
    models=[GradientBoostingRegressor(n_estimators=200)],
    split_y=True) 
# 4 
gp_uni = ModelsUnion(models=[gp.GaussianProcessRegressor(kernel=KERNEL_MAUNA, alpha=0, n_restarts_optimizer=20, normalize_y=True)], split_y=True)

# 5
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_uni = ModelsUnion(models=[svr_rbf], split_y=True)

# 6
mlp_reg = MLPRegressor(activation='relu', solver='lbfgs')
mlp_uni = ModelsUnion(models=[mlp_reg], split_y=True)

### Prediction tutor

In [4]:
gen = SamplesGenerator(pro)
# tutor = make_pipeline(PredictTutor(pro.get_bounds(), portfolio=[grad_uni, lin_uni]))
cv_thr = '(test_r2 > 0.65)'
test_thr = '(ndf_surr_score > 0.6)'


tutor = PredictTutor(pro.get_bounds(), portfolio=[svr_uni, grad_uni], solver='moea_control', train_test_sp=0.2, cv_threshold = cv_thr, test_threshold=test_thr)

cv_res = []
iter_solution = []
x=0
while x < 20:
    x=x+1
    print("\n--- {}".format(x))
    X, y = gen.return_X_y()
    tutor.fit(X.values, y.values, cv=4)

    propos = tutor.predict(n=3, kind='stack')
    iter_solution.append(tutor.predict_proba(None))
    cv_res.append(tutor.cv_result)
    gen.update(list(propos), [pro.fitness(p).tolist() for p in propos])

INFO:root:Available 0. At least 10.0 samples are required
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:Initialization data generator
INFO:root:Available 3. At least 10.0 samples are required
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:In dataset add 3 new results
INFO:root:Available 6. At least 10.0 samples are required
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:In dataset add 3 new results
INFO:root:Available 9. At least 10.0 samples are required
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:In dataset add 3 new results
INFO:root:Split dataset. Validation set is 0.2%



--- 1

--- 2

--- 3

--- 4

--- 5


INFO:root:Stage-2: 1 surrogate(s) valid
INFO:root:Stage-3: 0 surrogate(s) pass surr score
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:In dataset add 3 new results
INFO:root:Split dataset. Validation set is 0.2%



--- 6


INFO:root:Fit: Prediction from sampling plan
INFO:root:There is no valid surrogates
INFO:root:Prediction from sampling plan
INFO:root:In dataset add 3 new results
INFO:root:Split dataset. Validation set is 0.2%


...

MOEA-Ctrl: Evolve SVR vs GradientBoostingRegressor by 100 population size in 100 generation

--- 20


INFO:root:Stage-2: 2 surrogate(s) valid
INFO:root:Stage-3: 2 surrogate(s) pass surr score


MOEA-Ctrl: Evolve SVR vs GradientBoostingRegressor by 100 population size in 100 generation


INFO:root:In dataset add 3 new results


MOEA-Ctrl: Evolve GradientBoostingRegressor vs SVR by 100 population size in 100 generation


In [5]:
def fit(**cv_params):
    print(cv_params['cv'] if 'cvddd' in cv_params else 'fff')
    
    
fit(cv=9)

fff


In [8]:
pd.read_csv('/home/valavanca/Documents/hal/Diploma/src/benchmark_results/mtutor_on_zdt_i0.gtu2vpp9nj.csv')

,problem_name,seed,problem_id,objectives,feature_dim,pred_count,eval_budget,surr_portfolio,solver,train_test_sp,...,pop_ndf_f,fevals,evolve_time,date,p_distance,hypervolume,ndf_space,ndf_size,error,final
0,zdt,42,4,2,2,10,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",nsga2,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
1,zdt,42,4,2,2,10,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",nsga2,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
2,zdt,42,4,2,2,10,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",nsga2,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
3,zdt,42,4,2,2,10,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",nsga2,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
4,zdt,42,4,2,2,10,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",nsga2,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,zdt,42,6,2,2,50,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",random,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
2912,zdt,42,6,2,2,50,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",random,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
2913,zdt,42,6,2,2,50,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",random,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False
2914,zdt,42,6,2,2,50,1000,"[GaussianProcessRegressor(alpha=1e-10, copy_X_...",random,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tutor loop: The truth value of an array with m...,False


In [15]:
None in (4, None)

True

___
### Results analysis

In [ ]:
X, y = gen.return_X_y()
ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(y.values)
ndf_in_total = 100*(len(ndf[0])/len(X))
y.reset_index(drop=True, inplace=True)

print("Evaluated {} point(s). Non-dominated solutions is {:.1f}% from all dataset".format(len(X), ndf_in_total))
print("Hypervolume: {:.2f}".format(pg.hypervolume(-y.loc[ndf[0],:].values).compute([0]*OBJ)))

In [ ]:
X_ndf = X.copy().reset_index()
X_ndf['ndf'] = 0
X_ndf.loc[ndf[0],['ndf']] = 1
px.scatter_matrix(X_ndf, template="presentation" , dimensions=list(X.columns), color='ndf', color_continuous_scale='Jet', title="Feature space").update_layout(coloraxis_showscale=False)

In [ ]:
y_ndf = y.copy().reset_index()
y_ndf['ndf'] = 0
y_ndf.loc[ndf[0],['ndf']] = 1
px.scatter_matrix(y_ndf, template="presentation", dimensions=list(y.columns), color='ndf', color_continuous_scale='Rainbow', title="Objective space").update_layout(coloraxis_showscale=False)

In [ ]:
ndf_val_score = [frame.iloc[0]['ndf_surr_score'] if 'ndf_surr_score' in frame.columns else None for frame in iter_solution]
val_score = [frame.iloc[0]['surr_score'] if 'surr_score' in frame.columns else None for frame in iter_solution]

x = np.arange(len(val_score))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=ndf_val_score,
                    mode='lines+markers',
                    name='ndf_val_score'))
fig.add_trace(go.Scatter(x=x, y=val_score,
                    mode='lines+markers',
                    name='val_score'))
fig.update_layout(title='R2 score on the valid hypothesis')
fig.show()

### *Plotting objectives and search space for the last iteration of tuning
- Red points are Pare front from the hypothesis.
- Blue points are non-dominated solutions from evaluated points. Set of the best solutions available for this iteration.

**if the dimension of problem and objective space is 2*

In [ ]:
if OBJ==2 and DIM == 2:
    # Plot last population from TutorModel   
    X, y = gen.return_X_y()
    X = X.reset_index()
    y = y.reset_index()
    
    best_experiments = go.Scatter(x=y.loc[ndf[0],:].f1.values,
                         y=y.loc[ndf[0],:].f2.values,
                         mode='markers',
                         name=u'Dataset. Not-dominated results ',
                         showlegend=False,
                         line=dict(color='blue')
                         )
    
    solver = tutor.solution.solver.values[0]
    fig = plot_mo(solver.problem, solver.population)
    fig.add_trace(best_experiments, row=1, col=1)
    plotly.offline.iplot(fig)

In [ ]:
px.scatter_matrix(y.copy().loc[ndf[0],:], template="presentation", dimensions=list(y.drop(columns=['index']).columns), color_continuous_scale='Redblue', title="Pareto front").update_layout(coloraxis_showscale=False)

In [ ]:
px.scatter_matrix(X.copy().loc[ndf[0],:], template="presentation" , dimensions=list(X.drop(columns=['index']).columns), color_continuous_scale='Jet', title="Pareto front in parameter space").update_layout(coloraxis_showscale=False)

In [ ]:
nsga = [6.6, 6.5, 6,3]
com = [3,25]


go.Figure(data=[
    go.Bar(name='NSGA2', x=['_300','_500','_5000'], y=[6.6, 6.5, 6.3]),
    go.Bar(name='NSGA2+surrogate', x=['_300','_500','_5000'], y=[6.25])
])